In [ ]:
from azure.ai.ml import Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import CommandComponent

# Definir el input y output como carpetas URI
input_data = Input(type="uri_folder", path="azureml://datastores/ejemplo")
processed_data = Output(type="uri_folder")
model_output = Output(type="uri_folder")

# Definir componentes (pasos del pipeline)
prepare_data_component = CommandComponent(
    name="prepare_data",
    inputs={"raw_data": input_data},
    outputs={"processed_data": processed_data},
    command="python prepare_data.py --raw_data ${{inputs.raw_data}} --processed_data ${{outputs.processed_data}}",
)

train_model_component = CommandComponent(
    name="train_model",
    inputs={"training_data": processed_data},
    outputs={"trained_model": model_output},
    command="python train_model.py --training_data ${{inputs.training_data}} --trained_model ${{outputs.trained_model}}",
)

# Definir el pipeline completo
@pipeline(name="data_processing_training_pipeline")
def my_pipeline():
    prepare_data_node = prepare_data_component(raw_data=input_data)
    train_model_node = train_model_component(training_data=prepare_data_node.outputs.processed_data)
    return {"model_output": train_model_node.outputs.trained_model}

# Compilar el pipeline
pipeline_instance = my_pipeline()
pipeline_instance.submit(experiment_name="training_pipeline_example")
